# A1111 Model Installer — SDXL

Installs **Stable Diffusion XL (SDXL)** models and related assets into the WebUI data directory:

- Base & Refiner → `/workspace/a1111/models/Stable-diffusion/`
- ControlNet XL → `/workspace/a1111/models/ControlNet/`
- ADetailer → `/workspace/a1111/models/adetailer/`
- ESRGAN Upscalers → `/workspace/a1111/models/ESRGAN/`

**How it works**
- Choose which items to install using checkboxes.
- Paste additional checkpoint URLs if desired.
- Re-running skips already-downloaded files.

> If a model requires login on Hugging Face, set `HF_TOKEN` in your RunPod template.

In [ ]:
# --- Paths / setup ---
from pathlib import Path
import os

ROOT = Path(os.environ.get("WEBUI_ROOT", "/workspace/a1111"))
CKPT_DIR = ROOT / "models/Stable-diffusion"
CN_DIR   = ROOT / "models/ControlNet"
AD_DIR   = ROOT / "models/adetailer"
ESRGAN_DIR = ROOT / "models/ESRGAN"

for d in (CKPT_DIR, CN_DIR, AD_DIR, ESRGAN_DIR):
    d.mkdir(parents=True, exist_ok=True)

print("Model directories ready:")
print(f"✓ Checkpoints : {CKPT_DIR}")
print(f"✓ ControlNet  : {CN_DIR}")
print(f"✓ ADetailer   : {AD_DIR}")
print(f"✓ ESRGAN      : {ESRGAN_DIR}")

In [ ]:
# --- Helper functions ---
import requests, shutil
from tqdm import tqdm
from huggingface_hub import hf_hub_download

def human(n: int) -> str:
    for u in ["B","KB","MB","GB"]:
        if n < 1024: return f"{n:.1f} {u}"
        n /= 1024
    return f"{n:.1f} TB"

def download_direct(url: str, dest: Path):
    dest.parent.mkdir(parents=True, exist_ok=True)
    if dest.exists():
        print("✔ Exists:", dest); return dest
    r = requests.get(url, stream=True)
    r.raise_for_status()
    total = int(r.headers.get('Content-Length', 0))
    tmp = dest.with_suffix(dest.suffix + '.part')
    with open(tmp, 'wb') as f, tqdm(total=total, unit='B', unit_scale=True, desc=dest.name, ncols=80) as bar:
        for chunk in r.iter_content(1024*1024):
            if chunk:
                f.write(chunk)
                bar.update(len(chunk))
    tmp.replace(dest)
    print(f"✔ Saved: {dest} ({human(dest.stat().st_size)})")
    return dest

def download_hf(repo, filename, dest_dir, dest_name=None):
    dest = dest_dir / (dest_name or filename)
    if dest.exists():
        print("✔ Exists:", dest)
        return dest
    print(f"→ HF: {repo}/{filename}")
    path = hf_hub_download(repo_id=repo, filename=filename, token=os.environ.get('HF_TOKEN'), resume_download=True)
    shutil.copy2(path, dest)
    print(f"✔ HF saved: {dest} ({human(dest.stat().st_size)})")
    return dest

In [ ]:
# --- Interactive UI ---
import ipywidgets as W
from IPython.display import display, Markdown

base_box = W.Checkbox(description="Install SDXL Base (stabilityai/stable-diffusion-xl-base-1.0)", value=True)
ref_box  = W.Checkbox(description="Install SDXL Refiner (stabilityai/stable-diffusion-xl-refiner-1.0)", value=True)
ad_box   = W.Checkbox(description="Install ADetailer face model", value=True)
esr_box  = W.Checkbox(description="Install 4x-UltraSharp ESRGAN", value=True)

cn_labels = [
    ("Canny XL", "diffusers/controlnet-canny-sdxl-1.0"),
    ("Depth XL", "diffusers/controlnet-depth-sdxl-1.0"),
    ("SoftEdge XL", "diffusers/controlnet-softedge-sdxl-1.0"),
    ("OpenPose XL", "diffusers/controlnet-openpose-sdxl-1.0")
]
prechecked = {"Canny XL","SoftEdge XL","OpenPose XL"}
cn_boxes = [W.Checkbox(description=l, value=(l in prechecked)) for l, _ in cn_labels]

ckpt_area = W.Textarea(
    placeholder="Paste SDXL checkpoint URLs (.safetensors, .ckpt), one per line.",
    layout=W.Layout(width="100%", height="120px")
)

button = W.Button(description="Download Selected", button_style="success", icon="download")
out = W.Output()

def run_download(_):
    out.clear_output()
    with out:
        errs = []
        if base_box.value:
            try:
                download_hf("stabilityai/stable-diffusion-xl-base-1.0", "sd_xl_base_1.0.safetensors", CKPT_DIR)
            except Exception as e:
                errs.append(("SDXL Base", e))
        if ref_box.value:
            try:
                download_hf("stabilityai/stable-diffusion-xl-refiner-1.0", "sd_xl_refiner_1.0.safetensors", CKPT_DIR)
            except Exception as e:
                errs.append(("SDXL Refiner", e))
        if ad_box.value:
            try:
                download_direct("https://huggingface.co/Bing-su/adetailer/resolve/main/face_yolov8n.pt", AD_DIR / "face_yolov8n.pt")
            except Exception as e:
                errs.append(("ADetailer", e))
        if esr_box.value:
            try:
                download_hf("Coyote-A/4x-UltraSharp", "4x-UltraSharp.pth", ESRGAN_DIR)
            except Exception as e:
                errs.append(("ESRGAN", e))
        for box, (label, repo) in zip(cn_boxes, cn_labels):
            if box.value:
                try:
                    download_hf(repo, "diffusion_pytorch_model.safetensors", CN_DIR, f"controlnet-{label.replace(' ','_').lower()}.safetensors")
                except Exception as e:
                    errs.append((label, e))
        for url in [u.strip() for u in ckpt_area.value.splitlines() if u.strip()]:
            try:
                name = url.split('/')[-1].split('?')[0]
                if not name.endswith(('.safetensors','.ckpt')):
                    name += '.safetensors'
                download_direct(url, CKPT_DIR / name)
            except Exception as e:
                errs.append((url, e))
        print("\n=== Summary ===")
        if errs:
            for name, e in errs:
                print(f"✖ {name}: {e}")
        else:
            print("✔ All downloads completed successfully.")

button.on_click(run_download)

display(Markdown("### Select Models to Download"))
display(ckpt_area)
display(base_box, ref_box, ad_box, esr_box)
display(Markdown("#### ControlNet (SDXL):"))
for c in cn_boxes:
    display(c)
display(button, out)

## Restart WebUI (required after installing)
Restarts A1111 under **supervisord** so new models are recognized.

In [ ]:
import subprocess, time, shlex
try:
    cmd = "supervisorctl -c /etc/supervisord.conf restart a1111"
    out = subprocess.run(shlex.split(cmd), capture_output=True, text=True)
    if out.returncode == 0 and "a1111:" in (out.stdout or ""):
        print(out.stdout.strip())
    else:
        print("Supervisorctl unavailable, killing process…")
        subprocess.run(["pkill", "-f", "launch.py"], check=False)
        time.sleep(5)
        print("✔ WebUI terminated, Supervisor will auto-restart it")
    print("Wait 10–15 seconds, then refresh WebUI (7860).")
except Exception as e:
    print("⚠️ Restart failed:", e)
    print("Manual command:")
    print("supervisorctl -c /etc/supervisord.conf restart a1111")